# Simple Classifier
In der vergangenen Übung haben Sie einiges über die Struktur der Daten gelernt. Nun sollen Sie dieses Wissen einsetzen um eine erste Vorhersage auf einem Testdatensatz zu treffen. Im folgenden werden die notwendigen Imports getätigt und das Dataset eingelesen.

### Imports

In [1]:
import numpy as np
import csv as csv
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import random
%matplotlib inline

### Daten einlesen

In [2]:
DATA_FILE = './Data/original_titanic.csv'
df = pd.read_csv(DATA_FILE, header=0)
len(df)
#df.info()

1309

### Datenlücken interpolieren

Gerne können Sie zum Füllen der Datenlücken auch Ihre Implementierung aus dem vorherigen Arbeitsblatt einsetzen.

In [3]:
def prepareData(df):
    df.loc[(((df.Sex == "male")   &(df.Survived == 0)) & (df.Pclass==1)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "male")   &(df.Survived == 0)) & (df.Pclass==1), "Age"].mean()
    df.loc[(((df.Sex == "male")   &(df.Survived == 0)) & (df.Pclass==2)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "male")   &(df.Survived == 0)) & (df.Pclass==2), "Age"].mean()
    df.loc[(((df.Sex == "male")   &(df.Survived == 0)) & (df.Pclass==3)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "male")   &(df.Survived == 0)) & (df.Pclass==3), "Age"].mean()
    
    df.loc[(((df.Sex == "male")   &(df.Survived == 1)) & (df.Pclass==1)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "male")   &(df.Survived == 1)) & (df.Pclass==1), "Age"].mean()
    df.loc[(((df.Sex == "male")   &(df.Survived == 1)) & (df.Pclass==2)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "male")   &(df.Survived == 1)) & (df.Pclass==2), "Age"].mean()
    df.loc[(((df.Sex == "male")   &(df.Survived == 1)) & (df.Pclass==3)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "male")   &(df.Survived == 1)) & (df.Pclass==3), "Age"].mean()
    
    
    df.loc[(((df.Sex == "female") &(df.Survived == 0)) & (df.Pclass==1)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "female") &(df.Survived == 0)) & (df.Pclass==1), "Age"].mean()
    df.loc[(((df.Sex == "female") &(df.Survived == 0)) & (df.Pclass==2)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "female") &(df.Survived == 0)) & (df.Pclass==2), "Age"].mean()
    df.loc[(((df.Sex == "female") &(df.Survived == 0)) & (df.Pclass==3)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "female") &(df.Survived == 0)) & (df.Pclass==3), "Age"].mean()
    
    df.loc[(((df.Sex == "female") &(df.Survived == 1)) & (df.Pclass==1)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "female") &(df.Survived == 1)) & (df.Pclass==1), "Age"].mean()
    df.loc[(((df.Sex == "female") &(df.Survived == 1)) & (df.Pclass==2)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "female") &(df.Survived == 1)) & (df.Pclass==2), "Age"].mean()
    df.loc[(((df.Sex == "female") &(df.Survived == 1)) & (df.Pclass==3)) & df.Age.isnull(),"Age"] =  df.loc[((df.Sex == "female") &(df.Survived == 1)) & (df.Pclass==3), "Age"].mean()
    return df
    
df = prepareData(df)

### Datensatz stochastisch verändern

Es wird der komplette Datensatz durchmischt. Finden Sie hierfür geeignete Methoden (Python, Numpy, Pandas). Geben Sie die ersten fünf Zeilen des durchmischten Datensatzes aus. Weisen Sie nach, dass lediglich die Reihenfolge der Daten geändert wurde und <b>nicht(!)</b> die Merkmals-Spalten.

In [4]:
#randomice order of entries
df_shuffled = df.sample(frac=1) 
#df.head(5)
#df_shuffled.head(5)

In [5]:
print(df.Age[(df.Survived == 1) & (df.Sex == 'female')].mean())
print(df_shuffled.Age[(df.Survived == 1) & (df.Sex == 'female')].mean())

29.13103414953052
29.131034149530446


### Aufteilung in Trainings- und Testdatensatz

Teilen Sie den stochastisch veränderten Datensatz in Trainings- und Testdatensatz mit der Verteilung von 80% (Train) zu 20% (Test) auf. Geben Sie die Länge der Datensätze aus.

In [6]:
#split data into train and test
df_train = df_shuffled[:int(len(df_shuffled)* 0.8)] 
print(len(df_train))
df_test  =  df_shuffled[int(len(df_shuffled)* 0.8):] 
print(len(df_test))


1047
262


### Performance von Machine Learing Algorithmen

Die Performance von Machine Learing Algorithmen kann unter anderem anhand der Korreklassifizierungsrate (engl. Accuracy) ermittelt werden. Diese wird über die Verwechslungsmatrix (engl. Confusion matrix) (siehe Abbildung) wie folgt definiert: <br>

TP = True Positive <br>
TN = True Negative <br>
FP = False Positive <br>
FN = False Negative <br>
ACC = Accuracy <br>

$ACC = \frac{(TP + TN)} {(TP+TN+FP+FN)}$

<img src="./Figures/Confusion-Matrix.png" alt="drawing" style="width:300px;"/>



### Implementierung des Algorithmus

Schreiben Sie eine Funktion um die Klasse *Survived*, mit ihren beiden Ausprägungen 0 und 1, anhand von einem oder mehreren Merkmalen vorherzusagen. Verwenden Sie dazu ein einfaches Modell in welchem Sie ihr bisheriges Wissen über den Datensatz einsetzen.

In [7]:
#calculate Survival probability overall similar people by Sex, Age and Class
def predict(row):
    found_rows = df_train[(df_train.Sex == row.Sex) & (df_train.Age == row.Age) & (df_train.Pclass == row.Pclass)]
    if len(found_rows) > 0:
        randomRow = found_rows.sample(n=1)
        return randomRow.iloc[0].Survived
    else:
        return random.randint(0, 1)


### Testen des Algorithmus

Testen Sie die predict() Methode mit den Datenobjekten aus dem Testdatensatz. Ermitteln sie hierzu die Korreklassifizierungsrate.

In [8]:
def getAcc():
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for index, row in df_test.iterrows():
        predicted = predict(row)
        actual = row["Survived"]

        if predicted == 1:
            if actual == 1:
                tp += 1
            else:  
                fp += 1
        else:
            if actual == 1:
                fn += 1
            else: 
                tn += 1

    
    return (tp + tn) / (tp + tn + fp +fn)

In [9]:
print(getAcc())

0.7290076335877863
